In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE NETWORK RULE brave_api_rule
    MODE = EGRESS
    TYPE = HOST_PORT
    VALUE_LIST = ('api.search.brave.com:443');

In [ ]:
CREATE OR REPLACE SECRET brave_api_key
    TYPE = GENERIC_STRING
    SECRET_STRING = '<your-brave-api-key>';

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION brave_access_integration
    ALLOWED_NETWORK_RULES = (brave_api_rule)
    ALLOWED_AUTHENTICATION_SECRETS = (brave_api_key)
    ENABLED = TRUE;

In [ ]:
GRANT USAGE ON INTEGRATION brave_access_integration TO ROLE PUBLIC;  -- Replace PUBLIC with your role if needed

In [ ]:
CREATE STAGE IF NOT EXISTS custom_tools;
GRANT USAGE ON STAGE custom_tools TO ROLE PUBLIC; -- Replace PUBLIC with your role if needed

In [ ]:
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import StringType
from snowflake.snowpark import Session

session.add_packages('requests')
database = session.get_current_database()
schema = session.get_current_schema()

@sproc(
    name="brave_search_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@custom_tools",  # Or your preferred stage
    packages=['snowflake-snowpark-python', 'requests'],
    external_access_integrations=['brave_access_integration'],
    secrets={'api_key': 'brave_api_key'}
)
def brave_search(session: Session, query: str) -> str:
    import requests
    import json
    import _snowflake
    
    # Retrieve the API token from the secret
    token = _snowflake.get_generic_secret_string('api_key')
    
    # Brave Search API details
    url = "https://api.search.brave.com/res/v1/web/search"
    params = {"q": query}
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": token
    }
    
    # Make the API call
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        return json.dumps(response.json())
    else:
        return json.dumps({"error": response.text, "status_code": response.status_code})

session.sql("show procedures like 'brave_search_sproc'").collect()

In [ ]:
call brave_search_sproc('What is Snowflake Intelligence?')